# Exercise 3 - SAKI

The purpose of this exercise is to develop an algorithm that optimizes the route that a robot takes for pick-up and storage of items in a warehouse.

There are the following constraints : 

* Size of warehouse is {1..3} x {1..3}
* There is separate start/stop position outside the 3x3 storage space
* The first position the robot can move into is always (1, 1)
* Robots can move to adjacent fields (but not diagonally)
* There are three types of items, identified by color (white, blue, red)


1. Introduction : Explication du concept de reinforcement learning et des différents élements caractérisant la formalisation d'un problème de RL.

2. Markov Decision Process

In [1]:
# We start by importing all the librairies necessary for ourn work.
import numpy as np
import pandas as pd

# Introduction

Reinforcement learning (RL) is learning what to do, how to map situations to actions, so as to maximize
a numerical reward signal. The learner is not told which actions to take, but instead must discover
which actions yield the most reward by trying them.

In short, we can say that reinforcement learning is learning by trying.

In RL there is no supervisor, only a reward signal or a real number that tells the agent how good or bad was his action. 
Feedback from the environment might be delayed over several time steps, it’s not necessarily instantaneous 
e.g. for the task of reaching a goal in a grid-world, the feedback might be at the end when the agent reaches the goal. 
The agent might spend some time exploring and wandering in the environment until it finally reaches the goal after a while to realize what were the good and bad actions it has taken.

In machine learning or supervised learning, we have a dataset that describes the environment to the algorithm and the right answers or actions to do when faced with a specific situation, and the algorithm tries to generalize from that data to new situations.

In RL the agent influences the environment through its actions which in turn affect the subsequent data it receives from the environment, it’s an active learning process.

In the problem, an agent is supposed to decide the best action to select based on his current state. When this step is repeated, the problem is known as a __Markov Decision Process__.


# Markov Decision Process

The Markov decision process, better known as MDP, is an approach in reinforcement learning to take decisions in a gridworld environment. A gridworld environment consists of states in the form of grids.

Markov decision processes formally describe an environment for reinforcement learning, where the environment is fully observable ie the current state completely characterises the process

Markov property : "The future is independent of the past given the present"

Thus, any reinforcement learning task composed of a set of states, actions, and rewards that follows the Markov property would be considered an MDP.

The solution to an MDP is called a policy and the objective is to find the optimal policy for that MDP task.

A Markov Decision Process (MDP) model contains:

* A set of possible world states S.
* A set of Models.
* A set of possible actions A.
* A real valued reward function R(s,a).
* A policy the solution of Markov Decision Process.

## 1. Actions

A is a set of all possible action. A(s) defines the set of actions that can be taken being in state S.

Notre robot peut soit recupérer, soit y déposer des objets dans l'entrepôt. Toutefois il est à noté que nous avons 3 types d'items différents ce qui nous fera des actions différentes en fonction de l'item choisi.

en sommes notre robot pourra éffectuer les différentes actions : 

* pick up or store red item --> pick_up_red, store_red
* pick up or store blue item --> pick_up_blue, store_blue
* pick up or store white item --> pick_up_white, store_white

In [2]:
all_actions = ['pick_up_red', 'pick_up_blue', 'pick_up_white', 'store_red', 'store_blue', 'store_white']


## 2. States

All the sources used for the realization of this article...

References :

<a href="https://towardsdatascience.com/reinforcement-learning-demystified-36c39c11ec14">Reinforcement Learning Demystified</a>

<a href="https://towardsdatascience.com/reinforcement-learning-an-introduction-to-the-concepts-applications-and-code-ced6fbfd882d">Reinforcement Learning: An Introduction to the Concepts, Applications and Code</a>

<a href="https://medium.com/coinmonks/implement-reinforcement-learning-using-markov-decision-process-tutorial-272012fdae51">Implement Reinforcement learning using Markov Decision Process</a>

<a href="http://www.cs.cmu.edu/~10601b/slides/MDP_RL.pdf">Markov Decision Process and Reinforcement Learning</a>

<a href="https://hub.packtpub.com/reinforcement-learning-mdp-markov-decision-process-tutorial/">Implement Reinforcement learning using Markov Decision Process [Tutorial]</a>

<a href="https://www.geeksforgeeks.org/markov-decision-process/">Markov Decision Process</a>

<a href="http://incompleteideas.net/book/bookdraft2017nov5.pdf">SB11 - Sutton Barto - Book - Reinforcement Learning.pdf</a>